In [1]:
import pandas as pd

train = pd.read_csv('./data/뉴스그룹분류경진대회/train.csv')

train.head()

,id,text,target
0,0,"\nThey were, and even if Washington might cons...",10
1,1,"We run ""SpaceNews & Views"" on our STAREACH BBS...",14
2,2,\n\n\nNot to worry. The Masons have been demo...,19
3,3,"Only Brendan McKay, or maybe ARF, would come t...",17
4,4,Help: I am running some sample problems from O...,5


In [2]:
train.shape

(9233, 3)

In [4]:
def check_missing_col(dataframe):
  missing_col = []
  for col in dataframe.columns:
    missing_values = sum(dataframe[col].isna())
    is_missing = True if missing_values >= 1 else False
    if is_missing:
      print(f'결측치 존재 컬럼 : {col}')
      print(f'해당 컬럼에 총 {missing_values} 개의 결측치 존재')
      missing_col.append([col, dataframe[col].dtype])
  if missing_col == []:
    print('결측치 없음')
  return missing_col

missing_col = check_missing_col(train)

결측치 없음


In [5]:
X = train.text
y = train.target

In [6]:
X.head()

0    \nThey were, and even if Washington might cons...
1    We run "SpaceNews & Views" on our STAREACH BBS...
2    \n\n\nNot to worry.  The Masons have been demo...
3    Only Brendan McKay, or maybe ARF, would come t...
4    Help: I am running some sample problems from O...
Name: text, dtype: object

In [7]:
y.head()

0    10
1    14
2    19
3    17
4     5
Name: target, dtype: int64

### CountVectorizer
텍스트와 같은 데이터는 머신러닝 알고리즘에 바로 입력할 수 없다.  
따라서 컴퓨터가 계산을 하기 위해 먼저 문장을 숫자형 값으로 바꾼 다음 계산하도록 해야 한다.  
이것을 **워드 임베딩** 이라고 한다.  
문장의 요소 하나하나를 토큰화하여 최소의 단위로 쪼갠다.  

In [8]:
from sklearn.feature_extraction.text import CountVectorizer

sample_vectorizer = CountVectorizer()

In [14]:
sample_text1 = ["Hello, my name is dacon and I am a data scientist!"]

sample_vectorizer.fit(sample_text1)

CountVectorizer()

In [15]:
print(sample_vectorizer.vocabulary_)

{'hello': 4, 'my': 6, 'name': 7, 'is': 5, 'dacon': 2, 'and': 1, 'am': 0, 'data': 3, 'scientist': 8}


In [16]:
sample_text2 = ['You are learning dacon data science']

sample_vector = sample_vectorizer.transform(sample_text2)
print(sample_vector.toarray())

[[0 0 1 1 0 0 0 0 0]]


In [17]:
sample_text3 = ["You are learning dacon data science with news data"]

sample_vector_2 = sample_vectorizer.transform(sample_text3)
print(sample_vector_2.toarray())

[[0 0 1 2 0 0 0 0 0]]


길이가 2 이하인 단어들과 콤마, 느낌표 같은 특수 문자를 의미가 적다고 판단하여 제외시킨다.  최종적으로 9개의 단어들이 Vocab에 포함되어 있다.  
이 단어들의 출현 빈도로 이루어진 벡터를 **BoW(Bag of Words)** 라고 한다.  
이 BoW를 만드는데 두 가지 과정이 있다.  

(1) 우선 각 단어에 고유한 정수 인덱스를 부여  
(2) 각 인덱스의 위치에 단어 토큰의 등장 횟수를 기록한 벡터를 만듦  

즉 데이터가 쌓일수록 단어가 등장하 횟수가 점점 학습되어 어떤 단어가 얼마나 등장했는지 기준으로 문서가 어떤 성격의 문서인지를 판단하기 용이하게 된다.  

In [18]:
vectorizer = CountVectorizer()
vectorizer.fit(X)
X = vectorizer.transform(X)

In [19]:
vectorizer.inverse_transform(X[0])

[array(['and', 'been', 'bust', 'complete', 'consider', 'druce', 'even',
        'goals', 'has', 'here', 'if', 'in', 'might', 'minute', 'only',
        'patty', 'rework', 'that', 'they', 'trade', 'utter', 'washington',
        'were'], dtype='<U81')]

위는 CountVectorizer로 학습 시킨 것으로 X는 자연어가 아닌 숫자형 벡터 데이터로 이루어져 있다.  
머신러닝 모델의 input값으로 적절한 데이터의 형식을 갖추게 되었으며 역변환을 통해 문장의 단어들을 확인할 수 있다.

### Logistic Regression

In [20]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(max_iter=500)
model.fit(X, y)

LogisticRegression(max_iter=500)

In [21]:
from sklearn.metrics import accuracy_score

y_pred = model.predict(X[0])
print('예측 라벨 : ', y_pred)
print('실제 라벨 : ', train.target[0])

예측 라벨 :  [10]
실제 라벨 :  10


In [22]:
test = pd.read_csv('./data/뉴스그룹분류경진대회/test.csv')
test.head()

,id,text
0,0,\nThe VL-IDE Adapter can be much faster then t...
1,1,\n\nYeah. In a fire that reportedly burned ho...
2,2,":Judge: ""I grant you immunity from whatever ma..."
3,3,"I, too, put a corbin seat on my Hawk. I got t..."
4,4,\n\nDo I ever!!!!!! After 2 years of having h...


In [23]:
test_X = test.text

test_X_vect = vectorizer.transform(test_X)

pred = model.predict(test_X_vect)
print(pred)

[ 3 16 11 ...  4  0  7]


In [24]:
submission = pd.read_csv("./data/뉴스그룹분류경진대회/sample_submission.csv")
submission.head()

,id,target
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0


In [25]:
submission["target"] = pred
submission.head()

,id,target
0,0,3
1,1,16
2,2,11
3,3,8
4,4,13


In [26]:
submission.to_csv("./data/뉴스그룹분류경진대회/submission.csv", index=False)